In [ ]:
import requests
import xml.etree.ElementTree as ET
import re
import os
from bs4 import BeautifulSoup

def PennTreeBank( folgerTag ):
    POS = folgerTag
    if re.search( r'av\-c', folgerTag ):
        POS = 'RBR'
    elif re.search( r'av\-s', folgerTag ):
        POS = 'RBS'
    elif re.search( r'av', folgerTag ):
        POS = 'RB'
        
    if re.search( r'j\-c', folgerTag ):
        POS = 'JJR'
    elif re.search( r'j\-s', folgerTag ):
        POS = 'JJS'
    elif re.search( r'j', folgerTag ):
        POS = 'JJ'
        
    if re.search( r'n1', folgerTag ):
        POS = 'NN'
    elif re.search( r'n2', folgerTag ):
        POS = 'NNS'

        
    if re.search( r'vvb', folgerTag ):
        POS = 'VB'
    elif re.search( r'vmb', folgerTag ):
        POS = 'MD'
    elif re.search( r'vvm', folgerTag ):
        POS = 'VBP'
    elif re.search( r'vv', folgerTag ):
        POS = 'VBD'
        
    elif re.search( r'crq', folgerTag ):
        POS = 'WP'
          
    if re.search( r'^d', folgerTag ):
        POS = 'DT'
    
    return POS
    
    
        

ns = {'t': 'http://www.tei-c.org/ns/1.0'}

def readXML( url ):

    response = requests.get(url)

    fileName = os.path.basename(url)
    fileName_POS = re.sub( '\.xml$' , '_POS.txt' , fileName )
    fileName_lemma = re.sub( '\.xml$' , '_lemma.txt' , fileName )

    pos = open( fileName_POS , 'w' , encoding = 'utf-8' )
    lemma = open( fileName_lemma , 'w' , encoding = 'utf-8' )


    if response:
        root = ET.fromstring(response.text)
        #print(root)
        sp = root.findall( 't:text//t:sp' , ns)

        for e in sp:
            
            posLine = ''
            lemmaLine = ''
            w = e.findall( './/t:w' , ns)
            for word in w:
                #print(w)
                #posLine += word.text + ' '
                token = word.text
                attributes = word.attrib
                for a in attributes:
                    #print( '@{}: {}'.format( a , attributes[a] ) )
                    if a == 'lemma':
                        attributes[a] = re.sub( r'[|]' , ' ',  attributes[a] )       
                        lemmaLine += attributes[a].lower() + ' '
                    if a == 'ana':
                        attributes[a] = re.sub( '#' , '' , attributes[a] )
                        if re.search( r'[|]', attributes[a]  ):
                            parts1 = re.split( r'[|]', token ) 
                            parts2 = re.split( r'[|]', attributes[a])
                            for w in range( len(parts1) ):
                                posLine += parts1[w].lower() + '/' + PennTreeBank( parts2[w] ) + ' '
                        else:
                            posLine += token.lower() + '/' + PennTreeBank( attributes[a] ) + ' '
            pos.write( posLine + '\n' )
            lemma.write( lemmaLine + '\n' )
    

    pos.close()
    lemma.close()
    


baseUrl = 'https://www.folgerdigitaltexts.org/download/'


response = requests.get( baseUrl + 'teisimple.html' )
print( response.status_code )

if response.status_code == 200:
    response.encoding = 'utf-8' 
    soup = BeautifulSoup( response.text , "lxml" )

                         
    links = soup.find_all("a")
                          

    for l in links:
        linktext = l.string
        url = l.get( "href")
        if re.search( r'xml$' , url ):
            print( baseUrl + url)
            readXML( baseUrl + url  )
else:
    print('File could not be downloaded!')